In [ ]:
import json
import glob
import re
import pandas as pd
from os import path
from statistics import mean
import seaborn as sns
import matplotlib.pyplot as plt

data_dir = '../out'

In [ ]:
all_files = glob.glob(path.join(data_dir, "gen_*.json"))
gen_pattern = 'gen_0*([0-9]*).json$'

def extract_gen_number(filename):
    return int(re.search(gen_pattern, filename).group(1))

all_files.sort(key=extract_gen_number)

In [ ]:
particle_dfs_list = []
meta_dfs_list = []
for filename in all_files:
    gen_number = extract_gen_number(filename)
    
    with open(filename) as f:
        data = json.load(f)
    new_p_df = pd.json_normalize(data['pop'], record_path=['normalised_particles'])
    new_p_df['generation'] = gen_number

    meta = {
        'generation': gen_number, 
        'tolerance': data['pop']['tolerance'], 
        'acceptance': data['pop']['acceptance']
    }
    new_m_df = pd.DataFrame(data=meta, index=[gen_number])

    meta_dfs_list.append(new_m_df)
    particle_dfs_list.append(new_p_df)

particle_df = pd.concat(particle_dfs_list)
particle_df = particle_df.reset_index()

meta_df = pd.concat(meta_dfs_list)
meta_df = meta_df.reset_index()
meta_df = meta_df.drop('index', axis=1)
print(meta_df)

particle_df['mean_score'] = particle_df['scores'].map(mean)
df = particle_df.drop('scores', axis=1)

In [ ]:
# fig, ax = plt.subplots(figsize=(12,6))
# score_max = max(particle_df['mean_score'])
# ax.set_xlim(0, score_max)
sns.barplot(
    data=meta_df, 
    x='generation', 
    y='tolerance',
).set(title='Tolerance')

In [ ]:
sns.set_theme()
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim(0, 1)

sns.kdeplot(
   data=particle_df, 
   x="parameters.heads", 
   hue='generation',
   fill=True, 
   palette="viridis_r",
   alpha=.1, 
   linewidth=1,
   bw_adjust=.8, 
   cut=0, 
).set(title='Posterior Heads')
sns.move_legend(ax, "upper left")

In [ ]:
#sns.set_theme()
fig, ax = plt.subplots(figsize=(12,6))
score_max = max(particle_df['mean_score'])
ax.set_xlim(0, score_max)
sns.kdeplot(
    data=particle_df, 
    x='mean_score', 
    hue='generation',
    fill=True, 
    palette="rocket_r",
    alpha=.1, 
    linewidth=1,
    bw_adjust=.8, 
    cut=0,
).set(title='Mean Score Distribution')